# Welcome!
Below, we will learn to implement and train a policy to play atari-pong, using only the pixels as input. We will use convolutional neural nets, multiprocessing, and pytorch to implement and train our policy. Let's get started!

In [1]:
# render ai gym environment
!pip install gymnasium[accept-rom-license]
!pip install gymnasium[atari]
import gymnasium as gym

!pip install progressbar
import progressbar as pb

import random as rand
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
%matplotlib inline

# install package for displaying animation
!pip install JSAnimation
from JSAnimation.IPython_display import display_animation
from IPython.display import display

import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using device: ",device) 

  Using cached AutoROM-0.4.2-py3-none-any.whl (16 kB)
  Using cached importlib_resources-5.4.0-py3-none-any.whl (28 kB)
  Using cached click-8.0.4-py3-none-any.whl (97 kB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached AutoROM.accept-rom-license-0.5.0.tar.gz (10 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\David\.conda\envs\drlnd\python.exe' 'C:\Users\David\.conda\envs\drlnd\lib\site-packages\pip' install --ignore-installed --no-user --prefix 'C:\Users\David\AppData\Local\Temp\pip-build-env-g_4q41za\overlay' --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- setuptools wheel requests tqdm libtorrent click 'importlib-resources; python_version < '"'"'3.9'"'"''
       cwd: None
  Complete output (10 lines):
    Using cached setuptools-59.6.0-py3-none-any.whl (952 kB)
    Using cached wheel-0.37.1-py2.py3-none-any.whl (35 kB)
    Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
    Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  ERROR: Could not find a version that satisfies the requirement libtorrent (from versions: none)
  ERROR: No matching distribution found for libtorrent
  ----------------------------------------
ERROR: Command errored out with exit statu

ERROR: Could not find a version that satisfies the requirement ale-py~=0.8.0; extra == "atari" (from gymnasium[atari]) (from versions: 0.6.0.dev20200207, 0.7rc0, 0.7rc1, 0.7rc2, 0.7rc3, 0.7rc4, 0.7, 0.7.1)
ERROR: No matching distribution found for ale-py~=0.8.0; extra == "atari" (from gymnasium[atari])
C:\Users\David\.conda\envs\drlnd\lib\site-packages\gymnasium\core.py:27: UserWarning: WARN: Gymnasium minimally supports python 3.6 as the python foundation not longer supports the version, please update your version to 3.7+
  "Gymnasium minimally supports python 3.6 as the python foundation not longer supports the version, please update your version to 3.7+"


  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12078 sha256=ee61fc812ac66f11ddc7f4fe35621d5dc9db15f923d5f564c00be28be0d4dd5b
  Stored in directory: c:\users\david\appdata\local\pip\cache\wheels\bc\13\93\a9bf6b3d3966e4af014b0dbef027fdea47393faf47e990349f
Successfully built progressbar


  Created wheel for JSAnimation: filename=JSAnimation-0.1-py3-none-any.whl size=11431 sha256=9c30e5da5d61f5e5e096712ae74242ebe1fade4dcebfc9cc3d0de8382d2582eb
  Stored in directory: c:\users\david\appdata\local\pip\cache\wheels\b2\85\7b\da07ebc1e3987743ee5f0fe6be2bd55fb45698f8ea91622691
Successfully built JSAnimation
using device:  cpu


In [ ]:
import pong_utils
from parallelEnv import parallelEnv

In [ ]:
[k for k in list(gym.envs.registry.keys()) if "Pong" in k]

In [ ]:
# PongDeterministic does not contain random frameskip
# so is faster to train than the vanilla Pong-v4 environment

env = gym.make('PongDeterministic-v4')#, render_mode='rgb_array', frameskip=1, full_action_space=False)
#env = gym.make('Pong-v4', render_mode='rgb_array', old_step_api=True)#, obs_type='grayscale'

print("List of available actions: ", env.unwrapped.get_action_meanings())

# we will only use the actions 'RIGHTFIRE' = 4 and 'LEFTFIRE" = 5
# the 'FIRE' part ensures that the game starts again after losing a life
# the actions are hard-coded in pong_utils.py
RIGHT=4
LEFT=5

# Preprocessing
To speed up training, we can simplify the input by cropping the images and use every other pixel



In [ ]:
#import matplotlib
#import matplotlib.pyplot as plt

# show what a preprocessed image looks like
_, _ = env.reset()
_, _, _, _, _ = env.step(0)
# get a frame after 20 steps
for _ in range(20):
    frame, _, _, _, _ = env.step(LEFT)

plt.subplot(1,2,1)
plt.imshow(frame)
plt.title('original image')

plt.subplot(1,2,2)
plt.title('preprocessed image')

# 80 x 80 black and white image
plt.imshow(pong_utils.preprocess_single(frame), cmap='Greys')
plt.show()



# Policy

## Exercise 1: Implement your policy
 
Here, we define our policy. The input is the stack of two different frames (which captures the movement), and the output is a number $P_{\rm right}$, the probability of moving left. Note that $P_{\rm left}= 1-P_{\rm right}$

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# set up a convolutional neural net
# the output is the probability of moving right
# P(left) = 1-P(right)
class WeakPolicy(nn.Module):

    def __init__(self):
        super(WeakPolicy, self).__init__()
        
        
    ########
    ## 
    ## Modify your neural network
    ##
    ########
        
        # 80x80 to outputsize x outputsize
        # outputsize = (inputsize - kernel_size + stride)/stride 
        # (round up if not an integer)

        # output = 20x20 here
        self.conv = nn.Conv2d(2, 1, kernel_size=4, stride=4)
        self.size=1*20*20
        
        # 1 fully connected layer
        self.fc = nn.Linear(self.size, 1)
        self.sig = nn.Sigmoid()
        
    def forward(self, x):
        
    ########
    ## 
    ## Modify your neural network
    ##
    ########
    
        x = F.relu(self.conv(x))
        # flatten the tensor
        x = x.view(-1,self.size)
        return self.sig(self.fc(x))

# run your own policy!
# policy=Policy().to(device)
policy=pong_utils.Policy().to(device)

# we use the adam optimizer with learning rate 2e-4
# optim.SGD is also possible
import torch.optim as optim
optimizer = optim.Adam(policy.parameters(), lr=1e-4)

# Game visualization
pong_utils contain a play function given the environment and a policy. An optional preprocess function can be supplied. Here we define a function that plays a game and shows learning progress

In [ ]:
env.spec

In [ ]:
from matplotlib.animation import FileMovieWriter as filemoviewriter

In [ ]:
pong_utils.play(env, policy, time=100)#, preprocess=pong_utils.preprocess_single) 
# try to add the option "preprocess=pong_utils.preprocess_single"
# to see what the agent sees

# Function Definitions
Here you will define key functions for training. 

## Exercise 2: write your own function for training
(what I call scalar function is the same as policy_loss up to a negative sign)

### PPO
Later on, you'll implement the PPO algorithm as well, and the scalar function is given by
$\frac{1}{T}\sum^T_t \min\left\{R_{t}^{\rm future}\frac{\pi_{\theta'}(a_t|s_t)}{\pi_{\theta}(a_t|s_t)},R_{t}^{\rm future}{\rm clip}_{\epsilon}\!\left(\frac{\pi_{\theta'}(a_t|s_t)}{\pi_{\theta}(a_t|s_t)}\right)\right\}$

the ${\rm clip}_\epsilon$ function is implemented in pytorch as ```torch.clamp(ratio, 1-epsilon, 1+epsilon)```

In [ ]:
def unclipped_surrogate(policy, old_probs, states, actions, rewards,
                      discount = 0.995, epsilon=0.1, beta=0.01):

    ########
    ## 
    ## WRITE YOUR OWN CODE HERE
    ##
    ########
    
    actions = torch.tensor(actions, dtype=torch.int8, device=device)

    # convert states to policy (or probability)
    new_probs = pong_utils.states_to_prob(policy, states)
    new_probs = torch.where(actions == pong_utils.RIGHT, new_probs, 1.0-new_probs)

    # include a regularization term
    # this steers new_policy towards 0.5
    # prevents policy to become exactly 0 or 1 helps exploration
    # add in 1.e-10 to avoid log(0) which gives nan
    entropy = -(new_probs*torch.log(old_probs+1.e-10)+ \
        (1.0-new_probs)*torch.log(1.0-old_probs+1.e-10))

    return torch.mean(beta*entropy)


In [ ]:
# clipped surrogate function
# similar as -policy_loss for REINFORCE, but for PPO
def clipped_surrogate(policy, old_probs, states, actions, rewards,
                      discount=0.995,
                      epsilon=0.1, beta=0.01):

    discount = discount**np.arange(len(rewards))
    rewards = np.asarray(rewards)*discount[:,np.newaxis]
    
    # convert rewards to future rewards
    rewards_future = rewards[::-1].cumsum(axis=0)[::-1]
    
    mean = np.mean(rewards_future, axis=1)
    std = np.std(rewards_future, axis=1) + 1.0e-10

    rewards_normalized = (rewards_future - mean[:,np.newaxis])/std[:,np.newaxis]
    
    # convert everything into pytorch tensors and move to gpu if available
    actions = torch.tensor(actions, dtype=torch.int8, device=device)
    old_probs = torch.tensor(old_probs, dtype=torch.float, device=device)
    rewards = torch.tensor(rewards_normalized, dtype=torch.float, device=device)

    # convert states to policy (or probability)
    new_probs = states_to_prob(policy, states)
    new_probs = torch.where(actions == RIGHT, new_probs, 1.0-new_probs)
    
    # ratio for clipping
    ratio = new_probs/old_probs

    # clipped function
    clip = torch.clamp(ratio, 1-epsilon, 1+epsilon)
    clipped_surrogate = torch.min(ratio*rewards, clip*rewards)

    # include a regularization term
    # this steers new_policy towards 0.5
    # add in 1.e-10 to avoid log(0) which gives nan
    entropy = -(new_probs*torch.log(old_probs+1.e-10)+ \
        (1.0-new_probs)*torch.log(1.0-old_probs+1.e-10))

    
    # this returns an average of all the entries of the tensor
    # effective computing L_sur^clip / T
    # averaged over time-step and number of trajectories
    # this is desirable because we have normalized our rewards
    return torch.mean(clipped_surrogate + beta*entropy)

# Training
We are now ready to train our policy!
WARNING: make sure to turn on GPU, which also enables multicore processing. It may take up to 45 minutes even with GPU enabled, otherwise it will take much longer!

In [ ]:

#import numpy as np
# keep track of how long training takes
# WARNING: running through all 800 episodes will take 30-45 minutes

# training loop max iterations
episode = 400

# widget bar to display progress
#!pip install progressbar
#import progressbar as pb
widget = ['training loop: ', pb.Percentage(), ' ', 
          pb.Bar(), ' ', pb.ETA() ]
timer = pb.ProgressBar(widgets=widget, maxval=episode).start()

envs = parallelEnv('PongDeterministic-v4', n=8, seed=1234)

discount_rate = .99
epsilon = 0.1
beta = .01
tmax = 320
SGD_epoch = 4

# keep track of progress
mean_rewards = []

for e in range(episode):

    # collect trajectories
    old_probs, states, actions, rewards = pong_utils.collect_trajectories(envs, policy, tmax=tmax)
        
    total_rewards = np.sum(rewards, axis=0)

    # gradient ascent step
    for _ in range(SGD_epoch):
        
        # uncomment to utilize your own clipped function!
        L = -clipped_surrogate(policy, old_probs, states, actions, rewards, epsilon=epsilon, beta=beta)

        #L = -pong_utils.clipped_surrogate(policy, old_probs, states, actions, rewards,
        #                                  epsilon=epsilon, beta=beta)
        optimizer.zero_grad()
        L.backward()
        optimizer.step()
        del L
    
    # the clipping parameter reduces as time goes on
    epsilon*=.999
    
    # the regulation term also reduces
    # this reduces exploration in later runs
    beta*=.995
    
    # get the average reward of the parallel environments
    mean_rewards.append(np.mean(total_rewards))
    
    # display some progress every 20 iterations
    if (e+1)%20 == 0 :
        print("Episode: {0:d}, score: {1:f}".format(e+1, np.mean(total_rewards)))
        print(total_rewards)
        
    # update progress widget bar
    timer.update(e+1)
    
timer.finish()

In [ ]:
pong_utils.play(env, policy, time=200) 

In [ ]:
# save your policy!
#torch.save(policy, 'pong_PPO.policy')

# load policy if needed
# policy = torch.load('PPO.policy')

# try and test out the solution 
# make sure GPU is enabled, otherwise loading will fail
# (the PPO verion can win more often than not)!
#
if device != 'cpu':
    policy_solution = torch.load('PPO_OG.policy')
    pong_utils.play(env, policy_solution, time=2000) 